# Complete Workflow with PanelBox v0.8.0

**Tutorial Completo**: Test Runners & Master Report

Este notebook demonstra o workflow completo do PanelBox v0.8.0, incluindo:
1. Setup e preparação dos dados
2. PanelExperiment e ajuste de modelos
3. ValidationTest runner (NEW in v0.8.0!)
4. ComparisonTest runner (NEW in v0.8.0!)
5. Residual diagnostics
6. Master Report generation (NEW in v0.8.0!)
7. Temas e customização

---

## 1. Setup e Importações

Primeiro, vamos importar as bibliotecas necessárias e verificar a versão do PanelBox.

In [1]:
import panelbox as pb
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print(f"PanelBox version: {pb.__version__}")
print(f"Expected: 0.8.0 or higher")

PanelBox version: 0.8.0
Expected: 0.8.0 or higher


## 2. Carregar e Explorar Dados

Vamos usar o dataset Grunfeld de investimento, incluído no PanelBox.

In [2]:
# Carregar dataset Grunfeld
data = pb.load_grunfeld()

print(f"Dataset shape: {data.shape}")
print(f"Panel structure: {data['firm'].nunique()} firms, {data['year'].nunique()} years")
print(f"\nVariables: {list(data.columns)}")

# Visualizar primeiras linhas
data.head(10)

Dataset shape: (200, 5)
Panel structure: 10 firms, 20 years

Variables: ['firm', 'year', 'invest', 'value', 'capital']


,firm,year,invest,value,capital
0,1,1935,317.6,3078.5,2.8
1,1,1936,391.8,4661.7,52.6
2,1,1937,410.6,5387.1,156.9
3,1,1938,257.7,2792.2,209.2
4,1,1939,330.8,4313.2,203.4
5,1,1940,461.2,4643.9,207.2
6,1,1941,512.0,4551.2,255.2
7,1,1942,448.0,3244.1,303.7
8,1,1943,499.6,4053.7,264.1
9,1,1944,547.5,4379.3,201.6


### Estatísticas Descritivas

In [3]:
data.describe()

,firm,year,invest,value,capital
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,5.500000,1944.500000,146.350650,1078.106500,183.755500
std,2.879489,5.780751,216.067479,1317.151735,288.250932
min,1.000000,1935.000000,12.930000,42.000000,0.800000
25%,3.000000,1939.750000,36.527500,210.775000,38.800000
50%,5.500000,1944.500000,54.380000,445.000000,96.250000
75%,8.000000,1949.250000,137.650000,1679.850000,194.400000
max,10.000000,1954.000000,1486.700000,6241.700000,2226.300000


## 3. Criar PanelExperiment

O **PanelExperiment** é a interface principal do PanelBox para gerenciar múltiplos modelos.

In [4]:
# Criar experimento
experiment = pb.PanelExperiment(
    data=data,
    formula="invest ~ value + capital",
    entity_col="firm",
    time_col="year"
)

print(experiment)

PanelExperiment(
  formula='invest ~ value + capital',
  n_obs=200,
  n_models=0,
  models=[none]
)


## 4. Ajustar Múltiplos Modelos

Vamos ajustar 3 modelos diferentes: Pooled OLS, Fixed Effects e Random Effects.

In [5]:
# Ajustar modelos individuais
experiment.fit_model('pooled_ols', name='ols')
experiment.fit_model('fixed_effects', name='fe')
experiment.fit_model('random_effects', name='re')

# Listar modelos ajustados
print(f"\nModelos ajustados: {experiment.list_models()}")
print(experiment)

Fitting pooled_ols model 'ols'...
✅ Model 'ols' fitted successfully
Fitting fixed_effects model 'fe'...
✅ Model 'fe' fitted successfully
Fitting random_effects model 're'...
✅ Model 're' fitted successfully

Modelos ajustados: ['ols', 'fe', 're']
PanelExperiment(
  formula='invest ~ value + capital',
  n_obs=200,
  n_models=3,
  models=[ols, fe, re]
)


### Resumo dos Modelos

In [6]:
# Ver resumo de cada modelo
for model_name in experiment.list_models():
    results = experiment.get_model(model_name)
    print(f"\n{'='*60}")
    print(f"Model: {model_name.upper()}")
    print(f"{'='*60}")
    print(f"R²: {results.rsquared:.4f}")
    print(f"Adj. R²: {getattr(results, 'rsquared_adj', results.rsquared):.4f}")
    if hasattr(results, 'aic'):
        print(f"AIC: {results.aic:.2f}")
    if hasattr(results, 'bic'):
        print(f"BIC: {results.bic:.2f}")


Model: OLS
R²: 0.8090
Adj. R²: 0.8071

Model: FE
R²: 0.7899
Adj. R²: 0.7776

Model: RE
R²: 0.8003
Adj. R²: 0.7983


## 5. ValidationTest Runner (NEW in v0.8.0!)

O **ValidationTest** runner permite rodar testes de validação com configs preset.

In [7]:
from panelbox.experiment.tests import ValidationTest

# Criar runner
validation_runner = ValidationTest()

print(f"Configs disponíveis: {list(validation_runner.CONFIGS.keys())}")
print(f"\nQuick tests: {validation_runner.CONFIGS['quick']}")
print(f"Basic tests: {validation_runner.CONFIGS['basic']}")
print(f"Full tests: {validation_runner.CONFIGS['full']}")

Configs disponíveis: ['quick', 'basic', 'full']

Quick tests: ['heteroskedasticity', 'autocorrelation']
Basic tests: ['heteroskedasticity', 'autocorrelation', 'normality']
Full tests: ['heteroskedasticity', 'autocorrelation', 'normality', 'hausman']


### Rodar Validação com Config 'Full'

In [8]:
# Validar modelo Fixed Effects
validation_result = experiment.validate_model('fe')

print(validation_result.summary())

MODEL VALIDATION REPORT

Model Information:
  Type:    Fixed Effects
  Formula: invest ~ value + capital
  N obs:   200
  N entities: 10

VALIDATION TESTS SUMMARY
Test                                Statistic    P-value    Result    
------------------------------------------------------------------------------

Serial Correlation Tests:
  Wooldridge                        54.909       0.0000     REJECT    
  Breusch-Godfrey                   3.625        0.0569     OK        
  Baltagi-Wu                        -2.868       0.0041     REJECT    

Heteroskedasticity Tests:
  Modified Wald                     215.672      0.0000     REJECT    
  Breusch-Pagan                     61.825       0.0000     REJECT    
  White                             72.282       0.0000     REJECT    

Cross-Sectional Dependence Tests:
  Pesaran CD                        1.099        0.2717     OK        
  Breusch-Pagan LM                  182.964      0.0000     REJECT    
  Frees                       

### Gerar Validation Report HTML

In [9]:
# Salvar report HTML
val_path = validation_result.save_html(
    'validation_report_v08.html',
    test_type='validation',
    theme='professional'
)

print(f"✅ Validation report saved to: {val_path}")

✅ Validation report saved to: validation_report_v08.html


## 6. ComparisonTest Runner (NEW in v0.8.0!)

O **ComparisonTest** runner compara múltiplos modelos automaticamente.

In [10]:
from panelbox.experiment.tests import ComparisonTest

# Criar runner
comparison_runner = ComparisonTest()
print(comparison_runner)

ComparisonTest()


### Comparar Todos os Modelos

In [11]:
# Comparar OLS, FE, RE
comparison_result = experiment.compare_models(['ols', 'fe', 're'])

print(comparison_result.summary())

MODEL COMPARISON SUMMARY

Models Compared: 3
Comparison Date: 2026-02-12 19:01:20

Comparison Metrics:
--------------------------------------------------------------------------------
Model                     R²         R² Adj     AIC          BIC         
--------------------------------------------------------------------------------
ols                       0.8090     0.8071     N/A          N/A         
fe                        0.7899     0.7776     N/A          N/A         
re                        0.8003     0.7983     N/A          N/A         
--------------------------------------------------------------------------------

Best Models by Metric:
--------------------------------------------------------------------------------
  • Highest R²: ols



### Identificar Melhor Modelo

In [12]:
# Melhor modelo por AIC (menor é melhor)
best_aic = comparison_result.best_model('aic', prefer_lower=True)
print(f"Best model by AIC: {best_aic}")

# Melhor modelo por R² ajustado (maior é melhor)
best_r2 = comparison_result.best_model('rsquared_adj', prefer_lower=False)
print(f"Best model by Adj. R²: {best_r2}")

Best model by AIC: None
Best model by Adj. R²: ols


### Gerar Comparison Report HTML

In [13]:
# Salvar comparison report
comp_path = comparison_result.save_html(
    'comparison_report_v08.html',
    test_type='comparison',
    theme='professional'
)

print(f"✅ Comparison report saved to: {comp_path}")

✅ Comparison report saved to: comparison_report_v08.html


## 7. Residual Diagnostics (v0.7.0)

Analisar os resíduos do melhor modelo.

In [14]:
# Analisar resíduos do modelo FE
residual_result = experiment.analyze_residuals('fe')

print(residual_result.summary())

Residual Diagnostic Analysis

Summary Statistics:
--------------------------------------------------
Observations:               200
Mean:                   -0.0000
Std. Deviation:         48.5008
Min:                  -189.5946
Max:                   200.9929
Skewness:                0.0221
Kurtosis:                4.3242

Diagnostic Tests:
--------------------------------------------------
Shapiro-Wilk (Normality):        W = 0.889, p = 0.000 ✗ FAIL
Jarque-Bera (Normality):         JB = 155.84, p = 0.000 ✗ FAIL
Durbin-Watson (Autocorrelation): DW = 1.008 (Positive autocorrelation)
Ljung-Box (Autocorrelation):     Q = 65.35, p = 0.000 ✗ FAIL

Interpretation:
--------------------------------------------------
✗ Residuals may not be normally distributed
✗ Autocorrelation may be present
⚠ Some model assumptions may be violated


### Testes Diagnósticos Individuais

In [15]:
# Shapiro-Wilk (normalidade)
stat, pvalue = residual_result.shapiro_test
print(f"Shapiro-Wilk Test:")
print(f"  Statistic: {stat:.6f}")
print(f"  P-value: {pvalue:.6f}")
print(f"  Interpretation: {'Normal' if pvalue > 0.05 else 'Not normal'}")

# Durbin-Watson (autocorrelação)
dw = residual_result.durbin_watson
print(f"\nDurbin-Watson Test:")
print(f"  Statistic: {dw:.6f}")
print(f"  Interpretation: {'No autocorrelation' if 1.5 < dw < 2.5 else 'Autocorrelation present'}")

# Jarque-Bera (normalidade)
stat, pvalue = residual_result.jarque_bera
print(f"\nJarque-Bera Test:")
print(f"  Statistic: {stat:.6f}")
print(f"  P-value: {pvalue:.6f}")

# Ljung-Box (serial correlation)
stat, pvalue = residual_result.ljung_box
print(f"\nLjung-Box Test (10 lags):")
print(f"  Statistic: {stat:.6f}")
print(f"  P-value: {pvalue:.6f}")

Shapiro-Wilk Test:
  Statistic: 0.888696
  P-value: 0.000000
  Interpretation: Not normal

Durbin-Watson Test:
  Statistic: 1.007948
  Interpretation: Autocorrelation present

Jarque-Bera Test:
  Statistic: 155.841347
  P-value: 0.000000

Ljung-Box Test (10 lags):
  Statistic: 65.345964
  P-value: 0.000000


### Gerar Residuals Report HTML

In [16]:
# Salvar residuals report
res_path = residual_result.save_html(
    'residuals_report_v08.html',
    test_type='residuals',
    theme='professional'
)

print(f"✅ Residuals report saved to: {res_path}")

✅ Residuals report saved to: residuals_report_v08.html


## 8. Master Report Generation (NEW in v0.8.0!)

O **Master Report** é um relatório abrangente que integra todos os sub-reports.

In [17]:
# Gerar master report
master_path = experiment.save_master_report(
    'master_report_v08.html',
    theme='professional',
    title='PanelBox v0.8.0 - Complete Analysis',
    reports=[
        {
            'type': 'validation',
            'title': 'Fixed Effects Validation',
            'description': 'Specification tests for FE model',
            'file_path': 'validation_report_v08.html'
        },
        {
            'type': 'comparison',
            'title': 'Model Comparison',
            'description': 'Compare Pooled OLS, FE, and RE models',
            'file_path': 'comparison_report_v08.html'
        },
        {
            'type': 'residuals',
            'title': 'Residual Diagnostics',
            'description': 'Diagnostic tests for FE model residuals',
            'file_path': 'residuals_report_v08.html'
        }
    ]
)

print(f"✅ Master report saved to: {master_path}")
print(f"\nOpen this file in your browser to see:")
print(f"  - Experiment overview")
print(f"  - Summary of all fitted models")
print(f"  - Navigation to validation, comparison, and residuals reports")

✅ Master report saved to: master_report_v08.html

Open this file in your browser to see:
  - Experiment overview
  - Summary of all fitted models
  - Navigation to validation, comparison, and residuals reports


## 9. Explorando Diferentes Temas

PanelBox oferece 3 temas profissionais: **professional**, **academic**, **presentation**.

### Theme: Academic

In [18]:
# Gerar report com tema acadêmico
validation_result.save_html(
    'validation_academic.html',
    test_type='validation',
    theme='academic',
    title='Validation Report - Academic Style'
)

print("✅ Academic theme report saved")

✅ Academic theme report saved


### Theme: Presentation

In [19]:
# Gerar report com tema para apresentação
comparison_result.save_html(
    'comparison_presentation.html',
    test_type='comparison',
    theme='presentation',
    title='Model Comparison - Presentation Style'
)

print("✅ Presentation theme report saved")

✅ Presentation theme report saved


## 10. Exportar para JSON

Todos os resultados podem ser exportados para JSON para análise programática.

In [20]:
# Exportar todos os resultados para JSON
validation_result.save_json('validation_v08.json')
comparison_result.save_json('comparison_v08.json')
residual_result.save_json('residuals_v08.json')

print("✅ All results exported to JSON")
print("  - validation_v08.json")
print("  - comparison_v08.json")
print("  - residuals_v08.json")

✅ All results exported to JSON
  - validation_v08.json
  - comparison_v08.json
  - residuals_v08.json


## 11. Workflow Completo Resumido

Aqui está todo o workflow em um único bloco:

In [21]:
# Workflow completo em ~10 linhas
import panelbox as pb

# 1. Load data and create experiment
data = pb.load_grunfeld()
experiment = pb.PanelExperiment(data, "invest ~ value + capital", "firm", "year")

# 2. Fit models
experiment.fit_all_models(names=['pooled', 'fe', 're'])

# 3. Generate all reports
experiment.validate_model('fe').save_html('val.html', test_type='validation')
experiment.compare_models(['pooled', 'fe', 're']).save_html('comp.html', test_type='comparison')
experiment.analyze_residuals('fe').save_html('res.html', test_type='residuals')

# 4. Generate master report
experiment.save_master_report('master.html', reports=[
    {'type': 'validation', 'title': 'Validation', 'file_path': 'val.html'},
    {'type': 'comparison', 'title': 'Comparison', 'file_path': 'comp.html'},
    {'type': 'residuals', 'title': 'Residuals', 'file_path': 'res.html'}
])

print("✅ Complete workflow executed!")

Fitting pooled_ols model 'pooled'...
✅ Model 'pooled' fitted successfully
Fitting fixed_effects model 'fe'...
✅ Model 'fe' fitted successfully
Fitting random_effects model 're'...
✅ Model 're' fitted successfully
✅ Complete workflow executed!


## 12. Resumo das Novidades v0.8.0

### ✨ Test Runners
- **ValidationTest**: Runner configurável com presets (quick, basic, full)
- **ComparisonTest**: Comparação automática de múltiplos modelos
- APIs limpas e intuitivas

### ✨ Master Report
- Relatório mestre com overview do experimento
- Resumo de todos os modelos ajustados
- Navegação para validation, comparison e residuals reports
- Quick start guide embutido
- Design responsivo

### ✨ Workflow Integration
- Pipeline completo end-to-end
- Integração perfeita entre componentes
- Exportação JSON para todos os resultados
- 3 temas profissionais

---

## 📚 Próximos Passos

1. **Explore os HTML reports** gerados
2. **Experimente diferentes temas** (professional, academic, presentation)
3. **Use seus próprios dados** - substitua o dataset Grunfeld
4. **Customize os reports** - adicione título, descrições personalizadas
5. **Exporte para JSON** - análise programática dos resultados

---

**Made with ❤️ using PanelBox v0.8.0**